In [1]:
import pandas as pd

Dataset_1 = pd.read_csv("./TrafficLabelling/Dataset1.csv")
Dataset_1

FileNotFoundError: [Errno 2] No such file or directory: './TrafficLabelling/Dataset1.csv'

In [ ]:
list(Dataset_1)
#Dataset_1[' Label'].nunique()
#assert(Dataset_1[' Label'].nunique()>1)

In [ ]:
assert(Dataset_1[' Label'].nunique()>1)
Dataset_1[' Label'].nunique()

In [ ]:
print(Dataset_1['Flow ID'].nunique())
print(Dataset_1[' Source IP'].nunique())
print(Dataset_1[' Source Port'].nunique())
print(Dataset_1[' Destination IP'].nunique())
print(Dataset_1[' Destination Port'].nunique())

Q3:
Considering that we have the ground truth, we are trying to predict a target variable on a finite dataset, and we don't have an environment, I choose supervised learning.

In [ ]:
Dataset_1[' Label'].unique()

In [ ]:
Dataset_1[' Label'].replace(['BENIGN', 'FTP-Patator', 'SSH-Patator'],[0,1,1],inplace=True)
print(Dataset_1[' Label'].unique())
Dataset_1[' Label']

In [ ]:
#q4
print("Balance:")
print(Dataset_1[Dataset_1[' Label']==0][' Label'].count(), "Benign")
print(Dataset_1[Dataset_1[' Label']==1][' Label'].count(), "Other")

In [ ]:
#most important port numbers for security
#https://cowbell.insure/blog/port-security-1/
"""15 Netstat
20/21 FTP
23 Telnet
25 SMTP
50/51 IPSec
53 DNS
67/68 BOOTP
69 TFTP
79/49 TACACS+
80 HTTP
88 Kerberos
110 POP3
111 Port Map
119 NNTP
123 NTP
137-139 NetBIOS
143 IMAP
161 SNMP
389 LDAP
445 SMB
500 IPSec/ISAKMP
520 RIP
546/547 DHCP
636 SLDAP
1512 WINS
1701 L2TP
1720 323
1723 PPTP
1812/13 RADIUS
3389 RDP
5004/5005 RTP
5060/5061 SIP"""

In [ ]:
strs = r"""15 Netstat
20
21 FTP
23 Telnet
25 SMTP
50
51 IPSec
53 DNS
67
68 BOOTP
69 TFTP
79
49 TACACS+
80 HTTP
88 Kerberos
110 POP3
111 Port Map
119 NNTP
123 NTP
137
138
139 NetBIOS
143 IMAP
161 SNMP
389 LDAP
445 SMB
500 IPSec/ISAKMP
520 RIP
546
547 DHCP
636 SLDAP
1512 WINS
1701 L2TP
1720 323
1723 PPTP
1812
13 RADIUS
3389 RDP
5004
5005 RTP
5060
5061 SIP"""
l =strs.split()
nums = []
for s in l:
    try:
        nums.append(int(s))
        
    except:
        pass
print(nums)

In [ ]:
dfs_sources = {a:[] for a in nums}
dfs_dests = {a:[] for a in nums}
dfs_sources['other_ports'] = []
dfs_dests['other_ports'] = []
for i in range(len(Dataset_1)):
    if Dataset_1[' Source Port'][i] in nums:
        dfs_sources['other_ports'].append(0)
    else:
        dfs_sources['other_ports'].append(1)
    if Dataset_1[' Destination Port'][i] in nums:
        dfs_dests['other_ports'].append(0)
    else:
        dfs_dests['other_ports'].append(1)
        
    for k in dfs_sources.keys():
        if k == 'other_ports':
            continue
        if Dataset_1[' Source Port'][i]==k:
            dfs_sources[k].append(1)
        else:
            dfs_sources[k].append(0)
        if Dataset_1[' Destination Port'][i]==k:
            dfs_dests[k].append(1)
        else:
            dfs_dests[k].append(0)

In [ ]:
print(len(dfs_dests[50]))

In [ ]:
for k in dfs_sources.keys():
    df = pd.DataFrame(dfs_sources[k],columns=['source port '+str(k)])
    Dataset_1= Dataset_1.join(df)
    df = pd.DataFrame(dfs_dests[k],columns=[' Destination Port'+str(k)])
    Dataset_1= Dataset_1.join(df)

In [ ]:
Dataset_1.drop(' Source Port',axis = 1, inplace=True)
Dataset_1.drop(' Destination Port',axis = 1, inplace=True)
Dataset_1

In [ ]:
from matplotlib.pyplot import plot
import numpy as np

In [ ]:
d = {x : Dataset_1[(Dataset_1[' Label']==1) & (Dataset_1[' Flow Duration'] == x)][' Label'].count()\
     for x in Dataset_1[' Flow Duration'].unique()}

In [ ]:
d

In [ ]:
#so most cyberattacks occur below 276 flow duration
x = []
y = []
for k in sorted(d.keys()):
    x.append(k)
    y.append(d[k])
    if(d[k]>10):
        print(k)
plot(x,y)

In [ ]:
d = {x : Dataset_1[(Dataset_1[' Label']==1) & (Dataset_1['Flow Bytes/s'] == x)][' Label'].count()\
     for x in Dataset_1['Flow Bytes/s'].unique()}

d

In [ ]:


#
x = []
y = []
for k in sorted(d.keys()):
    x.append(k)
    y.append(d[k])
    if(d[k]>100):
        print(k)
    
plot(x,y)



In [ ]:
#t = list(Dataset_1[' Timestamp'].unique())


In [ ]:
#t

In [ ]:
"""
times = []
for i in range(len(t)):
    date_time = t[i].split()
    print(date_time)
    time = date_time[1]
    hour_min = time.split(':')
    hour_min[0] = int(hour_min[0])
    if hour_min[1] == '00':
        hour_min[1] = 0
    hour_min[1] = int(hour_min[1])

    if int(hour_min[0]) < 8:
        hour_min[0] = int(hour_min[0])+12
    times.append(int(hour_min[1]) + hour_min[0]*60)
    """

In [ ]:
"""Dataset_1.drop([' Timestamp'],axis=1,inplace=True)
Dataset_1 = Dataset_1.join(pd.DataFrame(times, columns=['Timestamp']))
Dataset_1"""


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
Dataset_1.drop('Flow ID',axis=1,inplace=True)# Flow ID is derived from source and destination IPs

In [ ]:
#Dataset_1[' Timestamp'] = pd.to_datetime(Dataset_1[' Timestamp'])

In [ ]:
Dataset_1.drop(' Timestamp',axis=1,inplace=True)

In [ ]:
#Dataset_1[" Source IP"]=Dataset_1[" Source IP"].apply(str)
#Dataset_1[" Destination IP"]=Dataset_1[" Destination IP"].apply(str)
Dataset_1.drop(' Source IP',axis=1,inplace=True)
Dataset_1.drop(' Destination IP',axis=1,inplace=True)

In [ ]:
Dataset_1

In [ ]:
nanList = []
for name in list(Dataset_1):
    if Dataset_1[name].isnull().values.any():
        nanList.append(name)
nanList


In [ ]:
Dataset_1['Flow Bytes/s'].isnull().sum()

In [ ]:
Dataset_1.replace([np.inf, -np.inf], np.nan, inplace=True)

In [ ]:
Dataset_1.dropna(inplace=True)

In [ ]:
Dataset_1['Flow Bytes/s'].isnull().sum()

In [ ]:
y = Dataset_1[' Label']
X = Dataset_1.drop(' Label',axis=1,inplace=False)#didn't lose too much data, less than .1%

In [ ]:
clf = DecisionTreeClassifier(random_state=0)

In [ ]:


cross_val_score(clf, X, y, cv=10)

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp =MLPClassifier()

In [ ]:
cross_val_score(mlp, X, y, cv=10)

In [ ]:
from sklearn import svm
s = svm.SVC()

In [ ]:
cross_val_score(s, X, y, cv=10)

In [ ]:
#q8
def function(filepath, clf,clustering = False):
    import pandas as pd

    Dataset_1 = pd.read_csv(filepath)
    Dataset_1

    list(Dataset_1)
    #Dataset_1[' Label'].nunique()
    #assert(Dataset_1[' Label'].nunique()>1)

    
    Dataset_1[' Label'].nunique()

    print(Dataset_1['Flow ID'].nunique())
    print(Dataset_1[' Source IP'].nunique())
    print(Dataset_1[' Source Port'].nunique())
    print(Dataset_1[' Destination IP'].nunique())
    print(Dataset_1[' Destination Port'].nunique())



    Dataset_1[' Label'].unique()

    Dataset_1[' Label'].replace(['BENIGN', 'FTP-Patator', 'SSH-Patator'],[0,1,1],inplace=True)
    print(Dataset_1[' Label'].unique())
    Dataset_1[' Label']

    #q4
    print("Balance:")
    print(Dataset_1[Dataset_1[' Label']==0][' Label'].count(), "Benign")
    print(Dataset_1[Dataset_1[' Label']==1][' Label'].count(), "Other")

    

    strs = r"""15 Netstat
    20
    21 FTP
    23 Telnet
    25 SMTP
    50
    51 IPSec
    53 DNS
    67
    68 BOOTP
    69 TFTP
    79
    49 TACACS+
    80 HTTP
    88 Kerberos
    110 POP3
    111 Port Map
    119 NNTP
    123 NTP
    137
    138
    139 NetBIOS
    143 IMAP
    161 SNMP
    389 LDAP
    445 SMB
    500 IPSec/ISAKMP
    520 RIP
    546
    547 DHCP
    636 SLDAP
    1512 WINS
    1701 L2TP
    1720 323
    1723 PPTP
    1812
    13 RADIUS
    3389 RDP
    5004
    5005 RTP
    5060
    5061 SIP"""
    l =strs.split()
    nums = []
    for s in l:
        try:
            nums.append(int(s))

        except:
            pass
    print(nums)

    dfs_sources = {a:[] for a in nums}
    dfs_dests = {a:[] for a in nums}
    dfs_sources['other_ports'] = []
    dfs_dests['other_ports'] = []
    for i in range(len(Dataset_1)):
        if Dataset_1[' Source Port'][i] in nums:
            dfs_sources['other_ports'].append(0)
        else:
            dfs_sources['other_ports'].append(1)
        if Dataset_1[' Destination Port'][i] in nums:
            dfs_dests['other_ports'].append(0)
        else:
            dfs_dests['other_ports'].append(1)

        for k in dfs_sources.keys():
            if k == 'other_ports':
                continue
            if Dataset_1[' Source Port'][i]==k:
                dfs_sources[k].append(1)
            else:
                dfs_sources[k].append(0)
            if Dataset_1[' Destination Port'][i]==k:
                dfs_dests[k].append(1)
            else:
                dfs_dests[k].append(0)

    print(len(dfs_dests[50]))

    for k in dfs_sources.keys():
        df = pd.DataFrame(dfs_sources[k],columns=['source port '+str(k)])
        Dataset_1= Dataset_1.join(df)
        df = pd.DataFrame(dfs_dests[k],columns=[' Destination Port'+str(k)])
        Dataset_1= Dataset_1.join(df)

    Dataset_1.drop(' Source Port',axis = 1, inplace=True)
    Dataset_1.drop(' Destination Port',axis = 1, inplace=True)
    Dataset_1

    from matplotlib.pyplot import plot
    import numpy as np

    d = {x : Dataset_1[(Dataset_1[' Label']==1) & (Dataset_1[' Flow Duration'] == x)][' Label'].count()\
         for x in Dataset_1[' Flow Duration'].unique()}

    d
    if not clustering:

        #so most cyberattacks occur below 276 flow duration
        x = []
        y = []
        for k in sorted(d.keys()):
            x.append(k)
            y.append(d[k])
            if(d[k]>10):
                print(k)
        plot(x,y)

        d = {x : Dataset_1[(Dataset_1[' Label']==1) & (Dataset_1['Flow Bytes/s'] == x)][' Label'].count()\
             for x in Dataset_1['Flow Bytes/s'].unique()}

        d



        #
        x = []
        y = []
        for k in sorted(d.keys()):
            x.append(k)
            y.append(d[k])
            if(d[k]>100):
                print(k)

        plot(x,y)
    



    


    from sklearn.tree import DecisionTreeClassifier
    from sklearn.model_selection import cross_val_score

    from sklearn.model_selection import train_test_split

    Dataset_1.drop('Flow ID',axis=1,inplace=True)# Flow ID is derived from source and destination IPs

    #Dataset_1[' Timestamp'] = pd.to_datetime(Dataset_1[' Timestamp'])

    Dataset_1.drop(' Timestamp',axis=1,inplace=True)

    #Dataset_1[" Source IP"]=Dataset_1[" Source IP"].apply(str)
    #Dataset_1[" Destination IP"]=Dataset_1[" Destination IP"].apply(str)
    Dataset_1.drop(' Source IP',axis=1,inplace=True)
    Dataset_1.drop(' Destination IP',axis=1,inplace=True)

    Dataset_1

    nanList = []
    for name in list(Dataset_1):
        if Dataset_1[name].isnull().values.any():
            nanList.append(name)
    nanList


    Dataset_1['Flow Bytes/s'].isnull().sum()

    Dataset_1.replace([np.inf, -np.inf], np.nan, inplace=True)

    Dataset_1.dropna(inplace=True)#didn't lose too much data, less than .1%

    Dataset_1['Flow Bytes/s'].isnull().sum()
    
    if not clustering:
        y = Dataset_1[' Label']
        X = Dataset_1.drop(' Label',axis=1,inplace=False)
        print(cross_val_score(clf, X, y, cv=10))
    else:
        clf.fit(Dataset_1)
        print(clf.cluster_centers_)
        

In [ ]:
#q9
for i in range(1,8):
    print("dataset",i)
    path = f"./TrafficLabelling/Dataset{i:d}.csv"#renamed all the files
    function(path,DecisionTreeClassifier(random_state=0))

In [ ]:
#clustering for the one with all benign
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import contingency_matrix
from sklearn.metrics import adjusted_rand_score

In [ ]:
path = "./TrafficLabelling/DatasetMonday.csv"
km = KMeans(n_clusters=3)
function(path,km,True)

Q10
Even with knowing practically nothing about cybersecurity, I can say that the vast majority of cyberattacks occur with minimal flow duration and flow bytes, although I have no idea what those are.  I would be interested in what an expert has to say about attacks with 0 flow bytes.  It is good to do at least a little research to make the models more effective.  Decision tree gets 99% accuracy.  I wonder if a subject matter expert could beat that.